In [12]:
import random

In [13]:
# function of rooling dice
def roll_dice(n=5):
    results = []
    for i in range(n):
        results.append(random.randint(1,6))
    return results

In [14]:
def create_empty_scorecard():
    scores = {
        '1' : None,
        '2' : None,
        '3' : None,
        '4' : None,
        '5' : None,
        '6' : None,
        'three_of_a_kind' : None,
        'four_of_a_kind': None,
        'full_house': None,
        'four_straight' : None,
        'five_straight' : None,
        'yahtzee' : None,
        'chance' : None       
    }
    return scores

In [15]:
def select_keep(dice):
    while True:
        try:
            dice_to_keep = input('Which dice to keep?')
            dice_move_forward = []
            dummy_dice = dice.copy() # here we create a copy of the list dice, so we can eliminate items in the future for loop without changing the original list
            dice_to_keep = dice_to_keep.replace(' ', '')
            for i in dice_to_keep:
                i = int(i)
                if i in dummy_dice:      
                    dice_move_forward.append(i)
                    dummy_dice.remove(i)
                else:
                    raise ValueError('Not Valid Selection')
            return dice_move_forward

        except ValueError as e:
            print(f'{e} Try again.' )

In [16]:
def reroll(dice, kept):
    for i in range(len(dice) - len(kept)):
        kept.append(random.randint(1,6))
    return kept    

In [17]:
def has_straight(dice, length):
    dummy_dice = dice.copy()
    dummy_dice = set(dummy_dice)
    dummy_dice = list(dummy_dice)
    dummy_dice.sort()
    is_there_sequence = 0
    for idx, i in enumerate(dummy_dice):
        check_length = list(range(i, i + length, 1))
        if check_length == dummy_dice[idx: idx + length]:
            is_there_sequence += 1
    if is_there_sequence > 0:
        return True
    else:
        return False

In [18]:
def evaluate(dice):
    '''
    This function returns the score for each category based on the current dice roll. 
    It evaluates number categories (1-6), three/four of a kind, full house, straights, yahtzee and chance.   
    '''
    scores = create_empty_scorecard() #creates a temporary scorecard to evaluate the hand 
    
    total = sum(dice) #sum of all dice (to be used in several categories below)
    
    counts = {} #dictionary to count the occurrences of each value
    for i in dice:
        if i in counts:
            counts[i] += 1 #if already appears we increase the count by 1
        else:
            counts[i] = 1 #if its the first occurrence (start at 1)
            
    for i in range(1, 7):
        scores[str(i)] = dice.count(i) * i #multiplies each value (i) by its occurrences (dice.count(i))

    for value in counts.values(): 
        if value >= 5: #if a given number appears 5 times in the dice roll
            scores["yahtzee"] = 50 #Yahtzee: 5 of the same number 
            scores["four_of_a_kind"] = total #a given number appearing 5 times implies appearing 4 times
            scores["three_of_a_kind"] = total #a given number appearing 5 times implies appearing 3 times
        if value >= 4: #if a given number appears 4 times in the dice roll
            scores["four_of_a_kind"] = total #four_of_a_kind: 4 of the same number 
            scores["three_of_a_kind"] = total #a given number appearing 4 times implies appearing 3 times
        elif value >= 3: #if a given number appears 3 times in the dice roll
            scores["three_of_a_kind"] = total #three_of_a_kind: 3 of the same number
    
    if 2 in counts.values() and 3 in counts.values(): #having exactly 2 times of one number and 3 times of another 
        scores["full_house"] = 25 
    
    if has_straight(dice,5): #checking for a sequence of 5 using has_straight function above
            scores["four_straight"] = 30
            scores["five_straight"] = 40 #if there is a sequence of 5, there is also a sequence of 4
    elif has_straight(dice,4): #checking for a sequence of 4 using has_straight function above
        scores["four_straight"] = 30
    
    scores["chance"] = total #total of all dice (no restrictions)
    
    return scores

In [19]:
def choose_score(scores, used):
    
    while True:
        try:
            raw = input('Which one to fill?')
            raw = raw.lower().replace(' ', '')
            if raw in used.keys() and used[raw] == None:
                used[raw] = scores[raw] 
                return used
            elif raw in used.keys() and used[raw] != None:
                raise ValueError('Input already selected')
            else:
                raise ValueError('Input not valid check for spelling mistakes')
            
        except ValueError as e:
            print(f'{e} Try again.')        

In [20]:
def display_scorecard(card):
    upper_section_score = 0
    card_points = 0
            
    for key, value in card.items():
        if key in ('1', '2', '3', '4', '5', '6'):
            upper_section_score +=(value or 0)
    bonus = 35 if upper_section_score >= 63 else 0       
    for key, value in card.items():
        print(f'{key}: {value}')
        card_points += (value or 0)
    
    print(f'Upper Section Score: {upper_section_score}')
    print(f'Bonus: {bonus}')
    print(f"Total Score: {bonus+upper_section_score+card_points}")

In [21]:
def play_round(card):
    rolling_the_first_dice = roll_dice()
    print(f"Your current hand is:{rolling_the_first_dice}")
    evaluate_the_first_throw = evaluate(rolling_the_first_dice)
    print(f"Your current scorecard is:\n{evaluate_the_first_throw}")
    choosing_after_first_throw = select_keep(rolling_the_first_dice)
    first_reroll = reroll(rolling_the_first_dice,choosing_after_first_throw)
    print(f"Your current hand is:{first_reroll}")
    evaluate_the_second_throw = evaluate(first_reroll)
    print(f"Your current scorecard is:\n{evaluate_the_second_throw}")
    choosing_after_second_throw = select_keep(first_reroll)
    second_reroll = reroll(first_reroll, choosing_after_second_throw)
    print(f"Your final hand is:{second_reroll}")
    evaluate_the_third_throw = evaluate(second_reroll)
    print(f"Your final scorecard is:\n{evaluate_the_third_throw}")
    chosen_score = choose_score(evaluate_the_third_throw, card)
    display_scorecard(chosen_score)

test = create_empty_scorecard()

In [15]:
play_round(test)

Your current hand is:[4, 5, 6, 3, 4]
Your current scorecard is:
{'1': 0, '2': 0, '3': 3, '4': 8, '5': 5, '6': 6, 'three_of_a_kind': None, 'four_of_a_kind': None, 'full_house': None, 'four_straight': 30, 'five_straight': None, 'yahtzee': None, 'chance': 22}


Which dice to keep? 


Your current hand is:[2, 2, 2, 2, 4]
Your current scorecard is:
{'1': 0, '2': 8, '3': 0, '4': 4, '5': 0, '6': 0, 'three_of_a_kind': 12, 'four_of_a_kind': 12, 'full_house': None, 'four_straight': None, 'five_straight': None, 'yahtzee': None, 'chance': 12}


Which dice to keep? 


Your final hand is:[4, 1, 5, 6, 4]
Your final scorecard is:
{'1': 1, '2': 0, '3': 0, '4': 8, '5': 5, '6': 6, 'three_of_a_kind': None, 'four_of_a_kind': None, 'full_house': None, 'four_straight': None, 'five_straight': None, 'yahtzee': None, 'chance': 20}


...

Input already selected Try again.


Which one to fill? 2


Input already selected Try again.


Which one to fill? 4


1: 0
2: 0
3: None
4: 8
5: None
6: None
three_of_a_kind: None
four_of_a_kind: None
full_house: None
four_straight: None
five_straight: None
yahtzee: None
chance: None
Upper Section Score: 8
Bonus: 0
Total Score: 16
